In [1]:
%load_ext autoreload
%autoreload 2

import os
# import wandb
import errno
import deepcell

import tensorflow as tf
import numpy as np

from wandb.keras import WandbCallback

from deepcell.data.tracking import prepare_dataset
from deepcell.data.tracking import Track, concat_tracks
from deepcell.model_zoo.tracking import GNNTrackingModel
from deepcell.utils.tracking_utils import trks_stats, load_trks

from tensorflow_addons.optimizers import RectifiedAdam as RAdam

# from wandb.keras import WandbCallback
from tensorflow.keras.callbacks import CSVLogger

In [2]:
# Verify GPU count
from deepcell import train_utils
num_gpus = train_utils.count_gpus()
print('Training on {} GPUs'.format(num_gpus))

Training on 1 GPUs


In [3]:
# tf.config.experimental.get_memory_info('GPU:0')

In [4]:
# setup directories
ROOT_DIR = '/data/tracking_data'  # TODO: Change this! Usually a mounted volume

MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models'))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs'))
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
OUTPUT_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'nuc_tracking'))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR, OUTPUT_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [5]:
# Load and view stats on this file
filename = 'train.trks'
path = os.path.join('../trk_data/',filename)
trks_data = load_trks(path)

In [6]:
# all_tracks = Track(tracked_data=trks_data)
# track_info = concat_tracks([all_tracks])

In [6]:
dataset_sizes = os.path.abspath(os.path.join(ROOT_DIR, 'dataset_idxs_dvc.npy'))
dataset_indicies = np.load(dataset_sizes, allow_pickle=True).tolist()

In [7]:
# from deepcell.datasets.tracked import hek293

# filename = 'sample_tracking.trks'
# (X_train, y_train), (X_test, y_test) = hek293.load_tracked_data(filename)
# path = os.path.join(DATA_DIR, filename)
# trks_data = load_trks(path)
# all_tracks = Track(tracked_data=trks_data)
# track_info = concat_tracks([all_tracks])

In [8]:
"""
Functions for metrics
"""
def filter_and_flatten(y_true, y_pred):
    n_classes = tf.shape(y_true)[-1]
    new_shape = [-1, n_classes]
    y_true = tf.reshape(y_true, new_shape)
    y_pred = tf.reshape(y_pred, new_shape)

    # Mask out the padded cells
    y_true_reduced = tf.reduce_sum(y_true, axis=-1)
    good_loc = tf.where(y_true_reduced == 1)[:, 0]

    y_true = tf.gather(y_true, good_loc, axis=0)
    y_pred = tf.gather(y_pred, good_loc, axis=0)
    return y_true, y_pred


class Recall(tf.keras.metrics.Recall):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true, y_pred = filter_and_flatten(y_true, y_pred)
        super(Recall, self).update_state(y_true, y_pred, sample_weight)


class Precision(tf.keras.metrics.Precision):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true, y_pred = filter_and_flatten(y_true, y_pred)
        super(Precision, self).update_state(y_true, y_pred, sample_weight)


def loss_function(y_true, y_pred):
    y_true, y_pred = filter_and_flatten(y_true, y_pred)
    return deepcell.losses.weighted_categorical_crossentropy(
        y_true, y_pred,
        n_classes=tf.shape(y_true)[-1],
        axis=-1)

# Define optimizer
optimizer = RAdam(learning_rate=1e-3, clipnorm=0.001)

# Define the loss function
losses = {'temporal_adj_matrices': loss_function}

# Define metrics
metrics = [
    Recall(class_id=0, name='same_recall'),
    Recall(class_id=1, name='different_recall'),
    Recall(class_id=2, name='daughter_recall'),
    Precision(class_id=0, name='same_precision'),
    Precision(class_id=1, name='different_precision'),
    Precision(class_id=2, name='daughter_precision'),
]

In [16]:
seed = 1   # random seed for training/validation data split
batch_size = 1
track_length = 8  # only train on 8 frames at once
val_size = .20  # % of data saved as validation
test_size = .1  # % of data held out as a test set
n_epochs = 2  # number of training epochs

# steps_per_epoch = 1000
# validation_steps = 200
steps_per_epoch = 5
validation_steps = 2

translation_range = 512 #X_train.shape[-2]

n_layers = 1 # number of graph convolutions


# for i in range(len(dataset_indicies)):
i=0
new_data = {}
new_data['lineages'] = list(np.array(trks_data['lineages'])[dataset_indicies[i]])
new_data['X'] = trks_data['X'][dataset_indicies[i],...]
new_data['y'] = trks_data['y'][dataset_indicies[i],...]

ds_size = len(dataset_indicies[i])

print()
print('data idx', i, 'size', ds_size)
print()

all_tracks = Track(tracked_data=new_data)
track_info = concat_tracks([all_tracks])

# find maximum number of cells in any frame
max_cells = track_info['appearances'].shape[2]


train_data, val_data, test_data = prepare_dataset(
    track_info,
    rotation_range=180,
    translation_range=translation_range,
    seed=seed,
    val_size=val_size,
    test_size=test_size,
    batch_size=batch_size,
    track_length=track_length)


data idx 0 size 22



100%|██████████| 22/22 [00:42<00:00,  1.94s/it]


In [20]:
# seed = 1   # random seed for training/validation data split
# batch_size = 4
# track_length = 8  # only train on 8 frames at once
# val_size = .20  # % of data saved as validation
# test_size = .1  # % of data held out as a test set
# n_epochs = 1  # number of training epochs

# # steps_per_epoch = 1000
# # validation_steps = 200
# steps_per_epoch = 10
# validation_steps = 2

# translation_range = 512 #X_train.shape[-2]

# n_layers = 1 # number of graph convolutions

# model_name = 'graph_tracking_model_seed{}'.format(seed)
# model_path = os.path.join(MODEL_DIR, model_name)

# # for i in range(len(dataset_indicies)):
# i=0
# new_data = {}
# new_data['lineages'] = list(np.array(trks_data['lineages'])[dataset_indicies[i]])
# new_data['X'] = trks_data['X'][dataset_indicies[i],...]
# new_data['y'] = trks_data['y'][dataset_indicies[i],...]

# ds_size = len(dataset_indicies[i])

# print()
# print('data idx', i, 'size', ds_size)
# print()

# all_tracks = Track(tracked_data=new_data)
# track_info = concat_tracks([all_tracks])

# # find maximum number of cells in any frame
# max_cells = track_info['appearances'].shape[2]


# model_name = 'graph_tracking_model_seed{}'.format(seed)
# model_path = os.path.join(MODEL_DIR, model_name)


# train_data, val_data, test_data = prepare_dataset(
#     track_info,
#     rotation_range=180,
#     translation_range=translation_range,
#     seed=seed,
#     val_size=val_size,
#     test_size=test_size,
#     batch_size=batch_size,
#     track_length=track_length)



model_name = 'graph_tracking_model_seed{}'.format(seed)
model_path = os.path.join(MODEL_DIR, model_name)

graph_layer = 'se2c'  # type of graph convolution layer
# for graph_layer in ['se2t', 'gcn', 'se2c', 'gcs']:

tm = GNNTrackingModel(max_cells=max_cells, n_layers=n_layers, graph_layer=graph_layer)

# Compile model
tm.training_model.compile(optimizer=optimizer, loss=losses, metrics=metrics)

layer = graph_layer
ds_size = 5

train_log = os.path.join(ROOT_DIR, f'train_logs/training_log_{layer}_{ds_size}.csv')
csv_logger = CSVLogger(train_log)

# run = wandb.init(project='testing_new', reinit=True)
# wandb.run.name = layer+f'_datasize_{ds_size}'

# wandb.log({'metrics': metrics,
#             'losses': losses})

# Train the model
train_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        model_path, monitor='val_loss',
        save_best_only=True, verbose=1,
        save_weights_only=False),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, verbose=1,
        patience=3, min_lr=1e-7), csv_logger]


loss_history = tm.training_model.fit(
    train_data,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_data,
    validation_steps=validation_steps,
    epochs=n_epochs,
    verbose=1,
    callbacks=train_callbacks)

print()
print('finished training')
print()


# Save models for prediction
inf_path = os.path.join(MODEL_DIR, f'TrackingModelInf_{layer}_datasize_{ds_size}')
ne_path = os.path.join(MODEL_DIR, f'TrackingModelNE_{layer}_datasize_{ds_size}')

tm.inference_model.save(inf_path)
tm.neighborhood_encoder.save(ne_path)

# run.finish()

print()
print('finished this shit')
print()


Epoch 1/2
5/5 [==============================] - ETA: 0s - loss: 0.7199 - same_recall: 0.3873 - different_recall: 0.0554 - daughter_recall: 0.3333 - same_precision: 0.0060 - different_precision: 0.9827 - daughter_precision: 9.1943e-05WARNING:tensorflow:5 out of the last 11 calls to <function Model.make_test_function.<locals>.test_function at 0x7fa97a113a60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


5/5 [==============================] - 34s 2s/step - loss: 0.7199 - same_recall: 0.3873 - different_recall: 0.0554 - daughter_recall: 0.3333 - same_precision: 0.0060 - different_precision: 0.9827 - daughter_precision: 9.1943e-05 - val_loss: 0.9438 - val_same_recall: 0.6614 - val_different_recall: 0.0000e+00 - val_daughter_recall: 0.0000e+00 - val_same_precision: 0.0255 - val_different_precision: 0.0000e+00 - val_daughter_precision: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.94384, saving model to /data/tracking_data/models/graph_tracking_model_seed1


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /data/tracking_data/models/graph_tracking_model_seed1/assets


INFO:tensorflow:Assets written to: /data/tracking_data/models/graph_tracking_model_seed1/assets


...

INFO:tensorflow:Assets written to: /data/tracking_data/models/TrackingModelInf_se2c_datasize_5/assets


INFO:tensorflow:Assets written to: /data/tracking_data/models/TrackingModelInf_se2c_datasize_5/assets


INFO:tensorflow:Assets written to: /data/tracking_data/models/TrackingModelNE_se2c_datasize_5/assets


INFO:tensorflow:Assets written to: /data/tracking_data/models/TrackingModelNE_se2c_datasize_5/assets



finished this shit



In [13]:
import pandas as pd

In [21]:
tmp = pd.read_csv(train_log)

In [22]:
tmp.head()

,epoch,daughter_precision,daughter_recall,different_precision,different_recall,loss,lr,same_precision,same_recall,val_daughter_precision,val_daughter_recall,val_different_precision,val_different_recall,val_loss,val_same_precision,val_same_recall
0,0,0.000092,0.333333,0.982661,0.055404,0.719878,0.001,0.005998,0.387300,0.0,0.0,0.0,0.0,0.943835,0.025482,0.661417
1,1,0.000408,0.250000,0.975007,0.330657,0.490691,0.001,0.026163,0.297297,0.0,0.0,0.0,0.0,1.869506,0.013350,0.904762


In [ ]:
NV_GPU=2 nvidia-docker run -it --entrypoint python3 -v $PWD/deepcell:/usr/local/lib/python3.6/dist-packages/deepcell/ -v $PWD/deepcell:/deepcell -v $PWD/notebooks:/notebooks -v /$PWD/../data:/data -v /deepcell_data/users/uriah/data-registry/data/training/tracking-nuclear/:/trk_data $USER/equiv_networks:latest /notebooks/training/tracking/train_tracking_models.py > logfiles.txt 2>&1

In [5]:
seed = 1   # random seed for training/validation data split
batch_size = 4
track_length = 8  # only train on 8 frames at once
val_size = .20  # % of data saved as validation
test_size = .1  # % of data held out as a test set
n_epochs = 12  # number of training epochs

steps_per_epoch = 1000
validation_steps = 200

n_layers = 1  # number of graph convolutions
# graph_layer = 'se2c'  # type of graph convolution layer

translation_range = 512 #X_train.shape[-2]

model_name = 'graph_tracking_model_seed{}'.format(seed)
model_path = os.path.join(MODEL_DIR, model_name)

In [6]:
def filter_and_flatten(y_true, y_pred):
    n_classes = tf.shape(y_true)[-1]
    new_shape = [-1, n_classes]
    y_true = tf.reshape(y_true, new_shape)
    y_pred = tf.reshape(y_pred, new_shape)

    # Mask out the padded cells
    y_true_reduced = tf.reduce_sum(y_true, axis=-1)
    good_loc = tf.where(y_true_reduced == 1)[:, 0]

    y_true = tf.gather(y_true, good_loc, axis=0)
    y_pred = tf.gather(y_pred, good_loc, axis=0)
    return y_true, y_pred


class Recall(tf.keras.metrics.Recall):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true, y_pred = filter_and_flatten(y_true, y_pred)
        super(Recall, self).update_state(y_true, y_pred, sample_weight)


class Precision(tf.keras.metrics.Precision):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true, y_pred = filter_and_flatten(y_true, y_pred)
        super(Precision, self).update_state(y_true, y_pred, sample_weight)


def loss_function(y_true, y_pred):
    y_true, y_pred = filter_and_flatten(y_true, y_pred)
    return deepcell.losses.weighted_categorical_crossentropy(
        y_true, y_pred,
        n_classes=tf.shape(y_true)[-1],
        axis=-1)

In [7]:
from tensorflow_addons.optimizers import RectifiedAdam as RAdam


# Define optimizer
optimizer = RAdam(learning_rate=1e-3, clipnorm=0.001)

# Define the loss function
losses = {'temporal_adj_matrices': loss_function}

# Define metrics
metrics = [
    Recall(class_id=0, name='same_recall'),
    Recall(class_id=1, name='different_recall'),
    Recall(class_id=2, name='daughter_recall'),
    Precision(class_id=0, name='same_precision'),
    Precision(class_id=1, name='different_precision'),
    Precision(class_id=2, name='daughter_precision'),
]

In [10]:
for i in range(len(dataset_indicies)):

    new_data = {}
    new_data['lineages'] = list(np.array(trks_data['lineages'])[dataset_indicies[i]])
    new_data['X'] = trks_data['X'][dataset_indicies[i],...]
    new_data['y'] = trks_data['y'][dataset_indicies[i],...]
    
    ds_size = len(dataset_indicies[i])

    print()
    print('data idx', i, 'size', ds_size)
    print()

    all_tracks = Track(tracked_data=new_data)
    track_info = concat_tracks([all_tracks])

    max_cells = track_info['appearances'].shape[2]

    train_data, val_data, test_data = prepare_dataset(
        track_info,
        rotation_range=180,
        translation_range=translation_range,
        seed=seed,
        val_size=val_size,
        test_size=test_size,
        batch_size=batch_size,
        track_length=track_length)

    graph_layers = ['se2t', 'gcn', 'se2c', 'gcs']
    for layer in graph_layers:
        print()
        print(layer)
        print()

        run = wandb.init(project='cell_tracking', reinit=True)
        wandb.run.name = layer+f'_datasize_{ds_size}'

        tm = GNNTrackingModel(max_cells=max_cells, n_layers=n_layers, graph_layer=layer)

        wandb.log({'metrics': metrics,
                   'losses': losses})

        tm.training_model.compile(optimizer=optimizer, loss=losses, metrics=metrics)

        # Train the model
        train_callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                model_path, monitor='val_loss',
                save_best_only=True, verbose=1,
                save_weights_only=False),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.5, verbose=1,
                patience=3, min_lr=1e-7), WandbCallback()
        ]

        loss_history = tm.training_model.fit(
            train_data,
            steps_per_epoch=steps_per_epoch,
            validation_data=val_data,
            validation_steps=validation_steps,
            epochs=n_epochs,
            verbose=1,
            callbacks=train_callbacks)


        # Save models for prediction
        inf_path = os.path.join(MODEL_DIR, f'TrackingModelInf_{layer}_datasize_{ds_size}')
        ne_path = os.path.join(MODEL_DIR, f'TrackingModelNE_{layer}_datasize_{ds_size}')

        tm.inference_model.save(inf_path)
        tm.neighborhood_encoder.save(ne_path)

        run.finish()


data idx 0 size 22



100%|██████████| 22/22 [00:41<00:00,  1.89s/it]



se2t



Epoch 1/12
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [ ]:
from wandb.keras import WandbCallback

# Verify GPU count
from deepcell import train_utils
num_gpus = train_utils.count_gpus()
print('Training on {} GPUs'.format(num_gpus))

layer = graph_layer

run = wandb.init(project='cell_tracking', reinit=True)
wandb.run.name = layer+f'_datasize_{ds_size}'

wandb.log({'metrics': metrics,
            'losses': losses})

# Train the model
train_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        model_path, monitor='val_loss',
        save_best_only=True, verbose=1,
        save_weights_only=False),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, verbose=1,
        patience=3, min_lr=1e-7), WandbCallback()
]

loss_history = tm.training_model.fit(
    train_data,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_data,
    validation_steps=validation_steps,
    epochs=n_epochs,
    verbose=1,
    callbacks=train_callbacks)